In [ ]:
import numpy as np
import pandas as pd
from davood_ml_functions import *
import scipy.stats as st
from warnings import filterwarnings
filterwarnings("ignore")

In [53]:
path = ... # path to the main dataset
path = r"C:\Users\Davood\Desktop\Bootcamp\DivarEstate\clean_data.csv"
df = pd.read_csv(path , index_col = 0)

In [54]:
# residential = ["residential-sell" , "residential-rent" , "temporary-rent"]
# df = df.loc[df["cat2_slug"].isin(residential)]

In [55]:
lower = df["Y"].quantile(0.02)
upper = df["Y"].quantile(0.98)
con1 = df["Y"] >= lower
con2 = df["Y"] <= upper
df = df.loc[con1 & con2]

In [56]:
lux_options = ["has_barbecue" , "has_pool" , "has_jacuzzi" , "has_sauna" , ]
normal_options = ["has_balcony" , "has_elevator" , "has_warehouse" , "has_parking" ,
"has_warm_water_provider" , "has_heating_system" , "is_rebuilt" , "has_cooling_system" , 
"has_restroom" , "has_security_guard" , "has_water" , "has_electricity" , "has_gas"]

df["has_lux"] = (df.loc[:,lux_options].sum(axis = 1) > 0)
df["has_normal"] = (df.loc[:,normal_options].sum(axis = 1) > 0)
df = df[["has_lux" , "has_normal" , "Y"]]
df = df.dropna(subset = [ "Y"])

In [107]:
X = df["Y"]
X1 = df.loc[df["has_lux"] == False , "Y"] # rows with no luxury option
X2 = df.loc[df["has_lux"] == True , "Y"] # rows with at least 1 luxury option
print(len(X1))
print(len(X2))
print(50 * "-")
print(X1.mean() / 10 ** 10)
print(X2.mean() / 10 ** 10)

632071
4523
--------------------------------------------------
0.30023300556988886
0.4303967834677648


## Test if normally distributed

In [90]:
do_JB_test(X)
print(50 * "-")
stat , p = st.normaltest(X)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = 2.6636
K_hat = 11.4812
JB = 2660668.9975
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


## Try to convert it to normal

In [68]:
X_transformed = np.log1p(X)
X_log = (X_transformed - X_transformed.mean()) / X_transformed.std()
do_JB_test(X_log)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = -0.7382
K_hat = 3.6805
JB = 70106.1115
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.


In [82]:
X_sqrt = np.sqrt(X)
do_JB_test(X_sqrt)
print(50 * "-")
stat , p = st.normaltest(X_sqrt)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = 1.1183
K_hat = 4.0572
JB = 162331.0967
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


In [84]:
X_boxcox , _ = st.boxcox(X + 1e-6)
do_JB_test(X_boxcox)
print(50 * "-")
stat , p = st.normaltest(X_boxcox)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = -0.0465
K_hat = 2.4716
JB = 7634.8161
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


In [85]:
X_yeojohnson , _ = st.yeojohnson(X)
do_JB_test(X_yeojohnson)
print(50 * "-")
stat , p = st.normaltest(X_yeojohnson)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = -0.0465
K_hat = 2.4716
JB = 7634.8177
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


In [86]:
X_rank = st.rankdata(X)
do_JB_test(X_rank)
print(50 * "-")
stat , p = st.normaltest(X_rank)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = 0.0008
K_hat = 1.799
JB = 38257.5667
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


In [87]:
X_zscore = (X - X.mean()) / X.std(ddof = 1)
do_JB_test(X_zscore)
print(50 * "-")
stat , p = st.normaltest(X_zscore)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = 2.6636
K_hat = 11.4812
JB = 2660668.9975
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


In [91]:
X_minmax = (X - X.min()) / (X.max() - X.min())
do_JB_test(X_minmax)
print(50 * "-")
stat , p = st.normaltest(X_minmax)
print(p)

H0: Data IS normally distributed.
H1: Data is NOT normally distributed.
--------------------------------------------------
S_hat = 2.6636
K_hat = 11.4812
JB = 2660668.9975
Chi2_0.95_2 = 5.9915
--------------------------------------------------
P-value = 0.0%
--------------------------------------------------
Reject H0; the distribution at α = 0.05 is NOT normal.
--------------------------------------------------
0.0


## Check for luxury options

In [97]:
alpha = 0.05

n1 = X1.shape[0]
n2 = X2.shape[0]

n_boot = 10000
rng = np.random.default_rng()
boot_diffs = np.empty(n_boot)
for i in range(n_boot):
    s1 = rng.choice(X1.values , size = n1 , replace = True)
    s2 = rng.choice(X2.values , size = n2 , replace = True)
    boot_diffs[i] = s2.mean() - s1.mean()
p_boot = np.mean(boot_diffs <= 0)

print("H0: mean(X2) > mean(X1)")
print("H1: mean(X2) <= mean(X1)")
print(50 * "-")
print("bootstrap p-value:" , p_boot)

if p_boot < alpha:
    print("Reject H0: mean(X2) > mean(X1)")
else:
    print("Fail to reject H0")


H0: mean(X2) > mean(X1)
H1: mean(X2) <= mean(X1)
--------------------------------------------------
bootstrap p-value: 0.0
Reject H0: mean(X2) > mean(X1)


## Check for non-luxury (normal) options

In [106]:
con1 = df["has_normal"] == False
con2 = df["has_lux"] == False

Y = df["Y"]
Y1 = df.loc[con1 & con2 , "Y"] # rows with no normal option
Y2 = df.loc[(~con1) & con2 , "Y"] # rows with at least 1 normal option
print(len(Y1))
print(len(Y2))
print(50 * "-")
print(Y1.mean() / 10 ** 10)
print(Y2.mean() / 10 ** 10)

456
631615
--------------------------------------------------
0.11807511695877192
0.30036451605840897


In [104]:
alpha = 0.05

n1 = Y1.shape[0]
n2 = Y2.shape[0]

n_boot = 10000
rng = np.random.default_rng()
boot_diffs = np.empty(n_boot)
for i in range(n_boot):
    s1 = rng.choice(Y1.values , size = n1 , replace = True)
    s2 = rng.choice(Y2.values , size = n2 , replace = True)
    boot_diffs[i] = s2.mean() - s1.mean()
p_boot = np.mean(boot_diffs <= 0)

print("H0: mean(Y2) > mean(Y1)")
print("H1: mean(Y2) <= mean(Y1)")
print(50 * "-")
print("bootstrap p-value:" , p_boot)

if p_boot < alpha:
    print("Reject H0: mean(Y2) > mean(Y1)")
else:
    print("Fail to reject H0")

H0: mean(Y2) > mean(Y1)
H1: mean(Y2) <= mean(Y1)
--------------------------------------------------
bootstrap p-value: 0.0
Reject H0: mean(Y2) > mean(Y1)
